# Setting up

In [2]:
import requests
import pandas as pd

# USDA API Key
API_KEY = "GA0LAyk7zcLgEjKMdSfOIOl7GJmL4wRleIlflcfp"

# Base URL
BASE_URL = "https://api.fas.usda.gov"

# Parameters for requests
params = {"api_key": API_KEY}

In [3]:
# Get country information for PSD API
ENDPOINT = "/api/psd/countries"

# Senda a HTTP GET Request to the URL
response = requests.get(BASE_URL + ENDPOINT, params=params)

if response.status_code != 200:
    print(f"Error: {response.status_code}, {response.text}")

data = response.json()

countries = pd.DataFrame(data)
print(countries)

    countryCode            countryName regionCode gencCode
0            AF            Afghanistan        R12      AFG
1            T3            Africa, NEC        R15     None
2            AL                Albania        R16      ALB
3            AG                Algeria        R10      DZA
4            AO                 Angola        R11      AGO
..          ...                    ...        ...      ...
246          YE          Yemen (Sanaa)        R09     None
247          YI  Yugoslavia (>01/2001)        R16     None
248          YU    Yugoslavia (>05/92)        R16     None
249          ZA                 Zambia        R11      ZMB
250          RH               Zimbabwe        R11      ZWE

[251 rows x 4 columns]


In [4]:
    # Get country information for ESR API
    ENDPOINT = "/api/esr/countries"

    # Senda a HTTP GET Request to the URL
    response = requests.get(BASE_URL + ENDPOINT, params=params)

    if response.status_code != 200:
        print(f"Error: {response.status_code}, {response.text}")

    data = response.json()

    countries = pd.DataFrame(data)
    print(countries)

     countryCode countryName               countryDescription  regionId  \
0              1    EUROPEAN   EUROPEAN UNION - 27                    1   
1              2     UNKNOWN                          UNKNOWN        99   
2           1010    GREENLD   GREENLAND                              11   
3           1220    CANADA    CANADA                                 11   
4           1610    MIGUEL    ST. PIERRE AND MIQUELON                11   
..           ...         ...                              ...       ...   
206         9350    GUAM      GUAM                                    9   
207         9410    CAN EN I  CANTON AND ENDERBURY ISLANDS            9   
208         9510    AM SAMOA  AMERICAN SAMOA                          9   
209         9610    NMARIANA  NORTHERN MARIANA ISLANDS                9   
210         9800    US O IS   US MINOR OUTLYING ISLAND                9   

    gencCode  
0       None  
1        AX1  
2        GRL  
3        CAN  
4       None  
..       

In [5]:
# Get country information for ESR API
ENDPOINT = "/api/esr/commodities"

# Senda a HTTP GET Request to the URL
response = requests.get(BASE_URL + ENDPOINT, params=params)

if response.status_code != 200:
    print(f"Error: {response.status_code}, {response.text}")

data = response.json()

commodities = pd.DataFrame(data)
print(commodities)
commodities.columns = ["code", "name", "unit_id"]



    commodityCode                                      commodityName  unitId
0             101                                        Wheat - HRW       1
1             102                                        Wheat - SRW       1
2             103                                        Wheat - HRS       1
3             104                                      Wheat - White       1
4             105                                      Wheat - Durum       1
5             106                                      Wheat - Mixed       1
6             107                                          All Wheat       1
7             201                                     Wheat Products       1
8             301                                             Barley       1
9             401                                               Corn       1
10            501                                                Rye       1
11            601                                               Oats       1

In [6]:
print(commodities[commodities["name"].str.contains("wheat", case=False)])
wheat_code = commodities.iloc[6, 0]
print(wheat_code)

print(commodities[commodities["name"].str.contains("soybean", case=False)])
soybeans_code = commodities.iloc[13, 0]
print(soybeans_code)

print(commodities[commodities["name"].str.contains("corn", case=False)])
corn_code = commodities.iloc[9, 0]
print(corn_code)

   code            name  unit_id
0   101     Wheat - HRW        1
1   102     Wheat - SRW        1
2   103     Wheat - HRS        1
3   104   Wheat - White        1
4   105   Wheat - Durum        1
5   106   Wheat - Mixed        1
6   107       All Wheat        1
7   201  Wheat Products        1
107
    code                 name  unit_id
13   801             Soybeans        1
14   901  Soybean cake & meal        1
15   902          Soybean Oil        1
801
   code  name  unit_id
9   401  Corn        1
401


In [7]:
commodity_code = soybeans_code
market_year = "2026"
# China
country_code = "2050"
ENDPOINT = f"/api/esr/exports/commodityCode/{commodity_code}/countryCode/{country_code}/marketYear/{market_year}"

response = requests.get(BASE_URL + ENDPOINT, params=params)
if response.status_code != 200:
    print(f"Error: {response.status_code}, {response.text}")

data = response.json()
exports = pd.DataFrame(data)
print(exports)




Empty DataFrame
Columns: []
Index: []


In [8]:
import plotly.express as px

exports["weekEndingDate"] = pd.to_datetime(exports["weekEndingDate"])
november_data = exports[(exports["weekEndingDate"].dt.year == 2025) & (exports["weekEndingDate"].dt.month == 11)]
weekly_total_exports = november_data.groupby("weekEndingDate")["weeklyExports"].sum().reset_index()

figure1 = px.line(
    weekly_total_exports,
    x="weekEndingDate",        # column used for x-axis
    y="weeklyExports",         # column used for y-axis
    title="Total Weekly Exports - November 2025",
    markers=True,
    labels={                    # this is how you name the axes
        "weekEndingDate": "Week Ending Date",  # friendly x-axis title
        "weeklyExports": "Exports (tons)"      # friendly y-axis title
    }
)

figure1.show()

KeyError: 'weekEndingDate'

In [ ]:
data_2026 = exports[(exports["weekEndingDate"].dt.year == 2025)]
weekly_total_exports = data_2026.groupby("weekEndingDate")["weeklyExports"].sum().reset_index()
weekly_total_exports["weekEndingDate"] = weekly_total_exports["weekEndingDate"].dt.strftime("%Y-%m-%d")

figure2 = px.line(
    weekly_total_exports,
    x="weekEndingDate",        # column used for x-axis
    y="weeklyExports",         # column used for y-axis
    title="Total Weekly Exports - November 2025",
    markers=True,
    labels={                    # this is how you name the axes
        "weekEndingDate": "Week Ending Date",  # friendly x-axis title
        "weeklyExports": "Exports (tons)"      # friendly y-axis title
    }
)

figure2.show()
figure2.write_json("agdatavault.com/public/exports2025.json")

FileNotFoundError: [Errno 2] No such file or directory: 'agdatavault.com/public/exports2025.json'

In [ ]:
import pandas as pd

# -----------------------------
# Example: Load your exports data
# Replace with your actual CSV or DataFrame source
# -----------------------------
# For demonstration, let's assume you have a CSV:
# exports_df = pd.read_csv("weekly_exports.csv")

# If you already have the DataFrame in memory, just use it:
# exports_df = your_dataframe_here

# Example dummy data
data = {
    "commodityCode": [401, 401, 401, 401],
    "countryCode": [1220, 2010, 2050, 2],
    "countryName": ["Mexico", "Japan", "China", "European Union"]
}
exports_df = pd.DataFrame(data)

# -----------------------------
# Extract unique country codes
# -----------------------------
countries_df = exports_df[['countryCode', 'countryName']].drop_duplicates().reset_index(drop=True)

# -----------------------------
# Convert to dictionary: countryName -> countryCode
# -----------------------------
country_dict = pd.Series(countries_df.countryCode.values, index=countries_df.countryName).to_dict()

# -----------------------------
# Print all country codes
# -----------------------------
print("All ESR countries and codes:")
print(countries_df)

# -----------------------------
# Example lookup
# -----------------------------
country_to_lookup = "Philippines"
code = country_dict.get(country_to_lookup)
if code:
    print(f"\nESR country code for {country_to_lookup}: {code}")
else:
    print(f"\n{country_to_lookup} not found in the exports data")


All ESR countries and codes:
   countryCode     countryName
0         1220          Mexico
1         2010           Japan
2         2050           China
3            2  European Union

Philippines not found in the exports data


In [ ]:
import requests
import pandas as pd

# USDA API Key
API_KEY = "GA0LAyk7zcLgEjKMdSfOIOl7GJmL4wRleIlflcfp"

# Base URL
BASE_URL = "https://api.fas.usda.gov"

# Parameters for requests
params = {"api_key": API_KEY}

# PSD commodities endpoint
ENDPOINT = "/api/psd/commodities"

# Send GET request
response = requests.get(BASE_URL + ENDPOINT, params=params)

if response.status_code != 200:
    print(f"Error fetching PSD commodities: {response.status_code}, {response.text}")
else:
    data = response.json()
    
    # Handle if JSON is nested (sometimes wrapped in 'data')
    if isinstance(data, dict) and "data" in data:
        data = data["data"]
    
    # Convert to DataFrame
    psd_commodities = pd.DataFrame(data)
    
    # Print all columns and all rows (careful if it's very large!)
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 2000)
    
    display(psd_commodities)



,commodityCode,commodityName
0,0577400,"Almonds, Shelled Basis"
1,0011000,"Animal Numbers, Cattle"
2,0013000,"Animal Numbers, Swine"
3,0574000,"Apples, Fresh"
4,0430000,Barley
5,0579305,"Cherries (Sweet&Sour), Fresh"
6,0711100,"Coffee, Green"
7,0440000,Corn
8,2631000,Cotton
9,0230000,"Dairy, Butter"


In [ ]:
import requests
import pandas as pd

# USDA API Key
API_KEY = "GA0LAyk7zcLgEjKMdSfOIOl7GJmL4wRleIlflcfp"

# Base URL
BASE_URL = "https://api.fas.usda.gov"

# Endpoint for PSD countries
ENDPOINT = "/api/psd/countries"

# Request parameters
params = {"api_key": API_KEY}

# Make the GET request
response = requests.get(BASE_URL + ENDPOINT, params=params)

# Check response status
if response.status_code == 200:
    try:
        # Load JSON response
        data = response.json()
        
        # Convert to pandas DataFrame
        psd_countries = pd.DataFrame(data)
        print("PSD Country Codes:\n")
        display(psd_countries)
        
    except Exception as e:
        print("Error parsing JSON:", e)
else:
    print("Error fetching data. HTTP status:", response.status_code)
    print(response.text)


PSD Country Codes:



,countryCode,countryName,regionCode,gencCode
0,AF,Afghanistan,R12,AFG
1,T3,"Africa, NEC",R15,None
2,AL,Albania,R16,ALB
3,AG,Algeria,R10,DZA
4,AO,Angola,R11,AGO
5,AV,Anguilla,R02,AIA
6,AC,Antigua and Barbuda,R02,ATG
7,AR,Argentina,R04,ARG
8,AM,Armenia,R07,ARM
9,AA,Aruba,R02,ABW
